<a href="https://colab.research.google.com/github/advait1987/tf_gendervoiceclassify/blob/master/Gender_Voice_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
!pip install -q tensorflow==2.0.0-alpha0
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 79.9MB 390kB/s 
     |████████████████████████████████| 3.0MB 37.3MB/s 
     |████████████████████████████████| 61kB 23.6MB/s 
     |████████████████████████████████| 419kB 46.3MB/s 


In [10]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../tmp"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [0]:
url = 'https://github.com/advait1987/tf_gendervoiceclassify/raw/master/voice.csv'
dataframe = pd.read_csv(url)

## **Explore**

In [47]:
dataframe.head(2)

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,0.0,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,0.0,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male


In [53]:
from sklearn.preprocessing import LabelEncoder
gender_encoder = LabelEncoder()
dataframe['label'] = gender_encoder.fit_transform(dataframe['label'])
dataframe.head(2)

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,0.0,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,1
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,0.0,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,1


In [54]:
print("Total number of samples: {}".format(dataframe.shape[0]))
print("Number of male: {}".format(dataframe[dataframe.label == 'male'].shape[0]))
print("Number of female: {}".format(dataframe[dataframe.label == 'female'].shape[0]))

Total number of samples: 3168
Number of male: 0
Number of female: 0


/usr/local/lib/python3.6/dist-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


Split Data

In [55]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

2027 train examples
507 validation examples
634 test examples


**Create an input pipeline using tf.data**
This will enable us to use feature columns as a bridge to map from the columns in the Pandas dataframe to features 
used to train the model. If we were working with a very large CSV file (so large that it does not fit into memory), we would use tf.data to read it from disk directly. That is not covered in this tutorial.

In [0]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('label')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [57]:
# Test input pipeline
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['median'])
  print('A batch of targets:', label_batch )

Every feature: ['meanfreq', 'sd', 'median', 'Q25', 'Q75', 'IQR', 'skew', 'kurt', 'sp.ent', 'sfm', 'mode', 'centroid', 'meanfun', 'minfun', 'maxfun', 'meandom', 'mindom', 'maxdom', 'dfrange', 'modindx']
A batch of ages: tf.Tensor([0.24518919 0.17537634 0.19586207 0.17715818 0.19957895], shape=(5,), dtype=float64)
A batch of targets: tf.Tensor([0 0 1 1 0], shape=(5,), dtype=int32)


Choose which columns to use

In [58]:
feature_columns = []

# numeric cols
for header in list(feature_batch.keys()):
  feature_columns.append(feature_column.numeric_column(header))

# bucketized cols
#age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
#feature_columns.append(age_buckets)

# indicator cols
#thal = feature_column.categorical_column_with_vocabulary_list(
#      'thal', ['fixed', 'normal', 'reversible'])
#thal_one_hot = feature_column.indicator_column(thal)
#feature_columns.append(thal_one_hot)

# embedding cols
#thal_embedding = feature_column.embedding_column(thal, dimension=8)
#feature_columns.append(thal_embedding)

# crossed cols
#crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
#crossed_feature = feature_column.indicator_column(crossed_feature)
#feature_columns.append(crossed_feature)

#print (feature_columns)
dataframe.dtypes

meanfreq    float64
sd          float64
median      float64
Q25         float64
Q75         float64
IQR         float64
skew        float64
kurt        float64
sp.ent      float64
sfm         float64
mode        float64
centroid    float64
meanfun     float64
minfun      float64
maxfun      float64
meandom     float64
mindom      float64
maxdom      float64
dfrange     float64
modindx     float64
label         int64
dtype: object

**Create feature later**

In [0]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
#Earlier, we used a small batch size to demonstrate how feature columns worked. We create a new input pipeline with a larger batch size.
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

**Create, compile, and train the model**

In [87]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.90):
      print (logs.get('acc'))
      print("\nReached 90% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()   

model = tf.keras.Sequential([
  feature_layer,
  #tf.keras.layers.Conv1D(32, (3), input_shape=(20,1), activation='relu'),
  #tf.keras.layers.MaxPooling2D(2, 2),
  #tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])


model.fit(train_ds,
          validation_data=val_ds,
          epochs=50,
          callbacks=[callbacks])

Epoch 1/50
64/64 [==============================] - 2s 27ms/step - loss: 0.6981 - acc: 0.5033 - val_loss: 0.7296 - val_acc: 0.5483
Epoch 2/50
64/64 [==============================] - 1s 19ms/step - loss: 0.7098 - acc: 0.6100 - val_loss: 0.6815 - val_acc: 0.6036
Epoch 3/50
64/64 [==============================] - 1s 19ms/step - loss: 0.6153 - acc: 0.6637 - val_loss: 0.6196 - val_acc: 0.6686
Epoch 4/50
64/64 [==============================] - 1s 20ms/step - loss: 0.6537 - acc: 0.7037 - val_loss: 0.6151 - val_acc: 0.6746
Epoch 5/50
64/64 [==============================] - 1s 19ms/step - loss: 0.6381 - acc: 0.7258 - val_loss: 0.6308 - val_acc: 0.6726
Epoch 6/50
64/64 [==============================] - 1s 20ms/step - loss: 0.6211 - acc: 0.7291 - val_loss: 0.6191 - val_acc: 0.6903
Epoch 7/50
64/64 [==============================] - 1s 19ms/step - loss: 0.6075 - acc: 0.7331 - val_loss: 0.6323 - val_acc: 0.6903
Epoch 8/50
64/64 [==============================] - 1s 19ms/step - loss: 0.5916 - a

**Accuracy**

In [75]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

20/20 [==============================] - 0s 11ms/step - loss: 0.3626 - acc: 0.9290
Accuracy 0.9290221
